In [1]:
# main_snp.py
"""Main program for snp
Date: 23-July-2019
Ver: 1.0
"""

##### Main Program for snp
from z_helper import *
util.startLoop()

from chains_snp import *
from ohlcs import *
from sized_snp import *
from target_snp import *
from workout_snp import *

# from json
a = assign_var('snp') + assign_var('common')
for v in a:
    exec(v)

def ask_user():
    '''Asks the user for what needs to be done
    Arg: None
    Returns: 0 to 6 int'''
    # Get user input
    askmsg = "\nChoose from the following numbers:\n" + \
            "0) Prepare for morning trades\n" + \
            "1) Chain Generation\n" + \
            "2) OHLCs Generation\n" + \
            "3) Size the options\n" + \
            "4) Target preparation\n"+ \
            "5) Place morning trades\n" + \
            "6) Workout closing trades (DYNAMIC)\n" + \
            "...Or close window to abort\n\n"
    
    while True:
        try:
            ip = int(input(askmsg+'\n'))
        except ValueError:
            print("\nSorry, I didn't understand what you entered. Try again!\n")
            continue # Loop again
        if not ip in [0, 1, 2, 3, 4, 5, 6]:
            print(f"\n{ip} is a wrong number! Choose any number from 0 to 7 and press <Enter>\n")
        else:
            break # success and exit loop
    
    return ip

# delete data and log files
def delete_all_data():
    '''Deletes all data and log files
    Arg: None Return: None'''
    folderpaths = ["../data/snp/", "../data/log/"]

    for folder in folderpaths:
        for files in listdir(folder):
            file_path = path.join(folder, files)
            try:
                if path.isfile(file_path):
                    unlink(file_path)
            except Exception as e:
                print(e)
                
    return None

# generate ohlcs
def make_ohlcs(ib, df_chains):
    '''Makes OHLCs for the underlying symbols
    Args:
        (ib) as connection object
        (df_chains) as DataFrame chain
    Returns: df_ohlcs as DataFrame of ohlcs and pickles them'''
    id_sym = df_chains.set_index('undId').symbol.to_dict()

    df_ohlcs = ohlcs(ib, id_sym, fspath, logpath)

    df_ohlcs.to_pickle(fspath+'ohlcs.pkl')
    
    return df_ohlcs

# do all the functions
def do_all(ib):
    '''Does all the functions outlined
    Args: None
    Returns: None'''
    
    # delete all data and log files
    delete_all_data()
    print("Deleted all data and log files\n")
    
    # do all the functions
    df_chains=get_chains(ib)
    print("Got the chains\n")
    
    df_ohlcs=make_ohlcs(ib, df_chains)
    print("Made the OHLCs\n")
    
    df_sized=sized_snp(ib, df_chains, df_ohlcs)
    print("Sized the options\n")
    
    # Error in margins
    if len(df_sized[df_sized.margin.isnull()]) == len(df_sized):
        print("\nERROR: Margins unavailable. Please run sizing again!\n")
        return None
    
    df_targets = target_snp(ib, df_sized, blacklist, snp_assignment_limit)
    print("Build the targets\n")
    
    df_sell = df_targets
    df_buy = workout_snp(ib)

    return df_buy, df_sell

In [3]:
# userip.py
# the selecting user inputs
if __name__=='__main__':
    userip = ask_user()
    
    with get_connected('snp', 'live') as ib:
        if userip == 0: # Run all
            start = time.time()
            print("\nRunning ALL\n")
            do_all(ib)
            print(f"\nTook {codetime(time.time()-start)} to complete do_all\n")
            
        elif userip == 1: # Chain Generation
            start = time.time()
            print("\nGetting Chains\n")
            df_chains=get_chains(ib)
            print(f"\nGot option chains in {codetime(time.time()-start)}\n")
            
        elif userip == 2: # OHLC Generation
            start = time.time()
            print("\nGenerating OHLCs\n")
            df_chains = pd.read_pickle(fspath+'chains.pkl')
            df_ohlcs=make_ohlcs(ib, df_chains)
            print(f"\nOHLCs generated in {codetime(time.time()-start)}\n")
            
        elif userip == 3: # Size the options
            start = time.time()
            print("Sizing the options\n")
            df_chains = pd.read_pickle(fspath+'chains.pkl')
            df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
            df_sized=sized_snp(ib, df_chains, df_ohlcs)
            print(f"\nOptions sized in {codetime(time.time()-start)}\n")
            
        elif userip == 4: # Target prepration
            start = time.time()
            print("Preparing targets\n")
            df_chains = pd.read_pickle(fspath+'chains.pkl')
            df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
            df_sized = pd.read_pickle(fspath+'sized_snp.pkl')
            df_targets = target_snp(ib, df_sized, blacklist, snp_assignment_limit)
            print(f"\nMade SELL targets in {codetime(time.time()-start)}\n")
            
        elif userip == 5: # Place the morning trades
            start = time.time()
            print("Trading in the morning\n")
            df_chains = pd.read_pickle(fspath+'chains.pkl')
            df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
            df_sized = pd.read_pickle(fspath+'sized_snp.pkl')            
            df_targets = pd.read_pickle(fspath+'targets.pkl')
            
            # cancel all existing trades
            ib.reqGlobalCancel()
            
            df_buy = workout_snp(ib)
            
            buy_tb = buys(ib, df_buy, exchange)
            sell_tb = sells(ib, df_targets, exchange)
            
            # place the morning trades
            morning_trades = place_morning_trades(ib, sell_tb=sell_tb, buy_tb=buy_tb)
            print(f"\nCompleted morning trades in {codetime(time.time()-start)}\n")
            
        elif userip == 6: # Workout closing trades for new fills
            start = time.time()
            print("Closing new fills\n")
            df_buy = workout_snp(ib)
            
            if df_buy.empty:
                print("\nNothing to buy!\n")
            else:
                buy_tb = buys(ib, df_buy, exchange)
                doTrades(ib, buy_tb)
                print(f"\nFilled close BUY orders in {codetime(time.time()-start)}\n")


Choose from the following numbers:
0) Prepare for morning trades
1) Chain Generation
2) OHLCs Generation
3) Size the options
4) Target preparation
5) Place morning trades
6) Workout closing trades (DYNAMIC)
...Or close window to abort


6


Peer closed connection
clientId 0 already in use?
API connection failed: CancelledError()



Closing new fills



Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Error 10197, reqId 1182863: No market data during competing live session, contract: Contract(secType='OPT', conId=382097302, symbol='V', lastTradeDateOrContractMonth='20191025', strike=160.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='V     191025P00160000', tradingClass='V')
Error 10197, reqId 1182746: No market data during competing live session, contract: Contract(secType='OPT', conId=378315244, symbol='PEP', lastTradeDateOrContractMonth='20190927', strike=120.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='PEP   190927P00120000', tradingClass='PEP')
Error 10197, reqId 1182749: No market data during competing live session, contract: Contract(secType='OPT', conId=380562909, symbol='PEP', lastTradeDateOrContractMonth='20191004', strike=121.0, right='P', multiplier='100', exchange='SMART', currency='USD', loc

Error 10197, reqId 1182674: No market data during competing live session, contract: Contract(secType='OPT', conId=382994503, symbol='T', lastTradeDateOrContractMonth='20191101', strike=32.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='T     191101P00032000', tradingClass='T')
Error 10197, reqId 1182862: No market data during competing live session, contract: Contract(secType='OPT', conId=381273067, symbol='LLY', lastTradeDateOrContractMonth='20191011', strike=101.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='LLY   191011P00101000', tradingClass='LLY')
Error 10197, reqId 1182737: No market data during competing live session, contract: Contract(secType='OPT', conId=376451450, symbol='HD', lastTradeDateOrContractMonth='20191018', strike=250.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='HD    191018C00250000', tradingClass='HD')
Error 10197, reqId 1182673: No market data during competing live 

Error 10197, reqId 1182692: No market data during competing live session, contract: Contract(secType='OPT', conId=378634395, symbol='PM', lastTradeDateOrContractMonth='20190927', strike=60.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='PM    190927P00060000', tradingClass='PM')
Error 10197, reqId 1182710: No market data during competing live session, contract: Contract(secType='OPT', conId=378623181, symbol='BLK', lastTradeDateOrContractMonth='20190927', strike=490.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='BLK   190927C00490000', tradingClass='BLK')
Error 10197, reqId 1182701: No market data during competing live session, contract: Contract(secType='OPT', conId=357019400, symbol='PEP', lastTradeDateOrContractMonth='20190920', strike=125.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='PEP   190920P00125000', tradingClass='PEP')
Error 10197, reqId 1182680: No market data during competing l

Error 10197, reqId 1182716: No market data during competing live session, contract: Contract(secType='OPT', conId=379783456, symbol='LMT', lastTradeDateOrContractMonth='20191018', strike=310.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='LMT   191018P00310000', tradingClass='LMT')
Error 10197, reqId 1182677: No market data during competing live session, contract: Contract(secType='OPT', conId=381273181, symbol='MCD', lastTradeDateOrContractMonth='20191011', strike=202.5, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='MCD   191011P00202500', tradingClass='MCD')
Error 10197, reqId 1182712: No market data during competing live session, contract: Contract(secType='OPT', conId=378293543, symbol='MA', lastTradeDateOrContractMonth='20190927', strike=247.5, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='MA    190927P00247500', tradingClass='MA')
Error 10197, reqId 1182748: No market data during competing 

KeyError: "['close' 'rom'] not in index"

In [ ]:
df_targets = pd.read_pickle(fspath+'targets.pkl')

In [ ]:
with get_connected('snp', 'live') as ib:
    co_blks = trade_blocks(ib, df_targets, 'SELL', exchange)
    trades = doTrades(ib, co_blks)

In [ ]:
with get_connected('snp', 'live') as ib:
    buy_df = workout_snp(ib)

In [ ]:
with get_connected('snp', 'live') as ib:
    
    # get all the trades
    trades = ib.trades()

    if trades: # there is something in trades
        all_trades_df = util.df(t.contract for t in trades).join(util.df(t.orderStatus for t in trades)).join(util.df(t.order for t in trades), lsuffix='_')
        all_trades_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis='columns', inplace=True)
        trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 
                       'orderId', 'permId', 'action', 'totalQuantity', 'lmtPrice', 'status']
        trades_df = all_trades_df[trades_cols]

        # get the sell option trades which are open (SUBMITTED)
        df_open_sells = trades_df[(trades_df.action == 'SELL') & 
                  (trades_df.secType == 'OPT') &
                  (trades_df.status == 'Submitted')]

        # cancel the sell open orders
        sell_openords = [t.order for t in trades if t.order.orderId in list(df_open_sells.orderId)]
    #         canceld_sells = [ib.cancelOrder(order) for order in sell_openords]


In [ ]:
df_open_sells

In [ ]:
with get_connected('snp', 'live') as ib:
    ib.reqGlobalCancel()

In [ ]:
df_targets = pd.read_pickle(fspath+'targets.pkl')

In [ ]:
df_targets